In [230]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('coins.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

In [231]:
# Gaussian filtering
blur = cv.GaussianBlur(gray,(5,5),0)
ret, thresh = cv.threshold(blur,200,255,cv.THRESH_BINARY_INV)

In [232]:
cv.imshow('gray', gray)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('image_a.jpg', gray)

cv.imshow('thresh', thresh)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('image_b.jpg', thresh)

True

In [233]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 3)

close = cv.morphologyEx(thresh,cv.MORPH_CLOSE,kernel, iterations = 3)

# sure background area
sure_bg = cv.dilate(opening, kernel, iterations=3)

# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
dist_transformNormalize = np.uint8(cv.normalize(dist_transform, None, 0, 255, cv.NORM_MINMAX))

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg, sure_fg)

In [234]:
cv.imshow('image', close) 
cv.waitKey(0) 
cv.destroyAllWindows()
cv.imwrite('close.jpg', close)

True

In [235]:
cv.imshow('image', dist_transformNormalize) 
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('image_d.jpg', dist_transformNormalize)

cv.imshow('image', sure_bg)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('image_e.jpg', sure_bg)

cv.imshow('image', sure_fg)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('image_f.jpg', sure_fg)

True

In [236]:
# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,255]

In [237]:
markers = np.float32(markers)
markers = np.uint8(cv.normalize(markers, None, 0, 255, cv.NORM_MINMAX))
cv.imshow('marker', markers)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('image_h.jpg', markers)

cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('image_i.jpg', img)

True